In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Metadata

meta_nirs1 = pd.read_csv('NIRS_csv/CC_NIRS_all70dyads_40channels_sub1.csv')
meta_nirs2 = pd.read_csv('NIRS_csv/CC_NIRS_all70dyads_40channels_sub2.csv')
meta_nirs1.fillna(0, inplace=True)
meta_nirs2.fillna(0, inplace=True)
meta_nirs1.iloc[:, 2:] = meta_nirs1.iloc[:, 2:] 
meta_nirs2.iloc[:, 2:] = meta_nirs2.iloc[:, 2:] 

face_freq, mfcc_freq, nirs_freq = 147, 423, 25 #au and mfcc are calculated by normalizing on nirs_freq ~5.09/sec for 5 seconds

In [3]:
dyads = []
for s in meta_nirs1['Subject']:
    if s not in dyads:
        dyads.append(s)

In [4]:
def reshape_array(array, freq):
    shape = array.shape
    data_num = shape[0]//freq
    array_reshaped = array[:data_num * freq, :].reshape(data_num, freq, shape[1])
    return array_reshaped

### Data Reshaping 

In [6]:
face_arrays = []
#mfcc_arrays = []
nirs_arrays = []
subject_idx = []
time_idx = []

for dyad in dyads:
    print(dyad)
    if dyad not in (3030, 4720):
        openface1 = pd.read_csv('OpenFace_csv/'+str(dyad)[:-1]+'1_conv/'+str(dyad)[:-1]+'1_conv.csv')
        face1 = openface1.iloc[:,-35:]
        #mfcc1 = pd.read_csv('Videos_Exactly20min_csv/'+str(dyad)[:-1]+'1_conv_20min/MFCC.csv').T
        nirs1 = meta_nirs1[meta_nirs1['Subject']==dyad].iloc[:,2:]
    else:
        print(dyad)
    
    if dyad not in (3030, 3170, 3230, 4040, 4700, 4720, 4060):
        openface2 = pd.read_csv('OpenFace_csv/'+str(dyad)[:-1]+'2_conv/'+str(dyad)[:-1]+'2_conv.csv')
        face2 = openface2.iloc[:,-35:]
        #mfcc2 = pd.read_csv('Videos_Exactly20min_csv/'+str(dyad)[:-1]+'2_conv_20min/MFCC.csv').T
        nirs2 = meta_nirs2[meta_nirs2['Subject']==dyad].iloc[:,2:]
    else:
        print(dyad)

# If including MFCC (audio feature):
#     face_array1, mfcc_array1, nirs_array1 = face1.values, mfcc1.values, nirs1.values
#     face_array2, mfcc_array2, nirs_array2 = face2.values, mfcc2.values, nirs2.values
    
#     print(face1.shape, mfcc1.shape, nirs1.shape)
#     print(face2.shape, mfcc2.shape, nirs2.shape)  
    
#     face_reshaped1, mfcc_reshaped1, nirs_reshaped1 = reshape_array(face_array1, face_freq), reshape_array(mfcc_array1, mfcc_freq), reshape_array(nirs_array1, nirs_freq)
#     min_shape = min(face_reshaped1.shape[0], mfcc_reshaped1.shape[0], nirs_reshaped1.shape[0])
#     face_reshaped1, mfcc_reshaped1, nirs_reshaped1 = face_reshaped1[:min_shape], mfcc_reshaped1[:min_shape], nirs_reshaped1[:min_shape]
    
#     face_reshaped2, mfcc_reshaped2, nirs_reshaped2 = reshape_array(face_array2, face_freq), reshape_array(mfcc_array2, mfcc_freq), reshape_array(nirs_array2, nirs_freq)
#     min_shape = min(face_reshaped2.shape[0], mfcc_reshaped2.shape[0], nirs_reshaped2.shape[0])
#     face_reshaped2, mfcc_reshaped2, nirs_reshaped2 = face_reshaped2[:min_shape], mfcc_reshaped2[:min_shape], nirs_reshaped2[:min_shape]

#     print(face_reshaped1.shape, mfcc_reshaped1.shape, nirs_reshaped1.shape)
#     print(face_reshaped2.shape, mfcc_reshaped2.shape, nirs_reshaped2.shape)
    
    face_array1, nirs_array1 = face1.values, nirs1.values
    face_array2, nirs_array2 = face2.values, nirs2.values

    print(face1.shape, nirs1.shape)
    print(face2.shape, nirs2.shape)
    
    face_reshaped1, nirs_reshaped1 = reshape_array(face_array1, face_freq), reshape_array(nirs_array1, nirs_freq)
    min_shape = min(face_reshaped1.shape[0], nirs_reshaped1.shape[0])
    face_reshaped1, nirs_reshaped1 = face_reshaped1[:min_shape], nirs_reshaped1[:min_shape]
    
    face_reshaped2, nirs_reshaped2 = reshape_array(face_array2, face_freq), reshape_array(nirs_array2, nirs_freq)
    min_shape = min(face_reshaped2.shape[0], nirs_reshaped2.shape[0])
    face_reshaped2, nirs_reshaped2 = face_reshaped2[:min_shape], nirs_reshaped2[:min_shape]

    print(face_reshaped1.shape, nirs_reshaped1.shape)
    print(face_reshaped2.shape, nirs_reshaped2.shape)
    
    face_arrays.extend([face_reshaped1, face_reshaped2])
    #mfcc_arrays.extend([mfcc_reshaped1, mfcc_reshaped2])
    nirs_arrays.extend([nirs_reshaped1, nirs_reshaped2])
    
    subject_idx.extend([dyad+1]*face_reshaped1.shape[0]+[dyad+2]*face_reshaped2.shape[0])
    time_idx.extend([i for i in range(1, face_reshaped1.shape[0] + 1)]+[i for i in range(1, face_reshaped2.shape[0] + 1)])

3020
(36165, 35) (6104, 40)
(36180, 35) (6104, 40)
(244, 147, 35) (244, 25, 40)
(244, 147, 35) (244, 25, 40)
3030
3030
3030
(36165, 35) (6104, 40)
(36180, 35) (6104, 40)
(244, 147, 35) (244, 25, 40)
(244, 147, 35) (244, 25, 40)
3120
(36399, 35) (6104, 40)
(36442, 35) (6104, 40)
(244, 147, 35) (244, 25, 40)
(244, 147, 35) (244, 25, 40)
3130
(36321, 35) (6104, 40)
(36339, 35) (6104, 40)
(244, 147, 35) (244, 25, 40)
(244, 147, 35) (244, 25, 40)
3170
3170
(35559, 35) (6104, 40)
(36339, 35) (6104, 40)
(241, 147, 35) (241, 25, 40)
(244, 147, 35) (244, 25, 40)
3230
3230
(36166, 35) (6104, 40)
(36339, 35) (6104, 40)
(244, 147, 35) (244, 25, 40)
(244, 147, 35) (244, 25, 40)
4040
4040
(34518, 35) (6104, 40)
(36339, 35) (6104, 40)
(234, 147, 35) (234, 25, 40)
(244, 147, 35) (244, 25, 40)
4050
(37844, 35) (6104, 40)
(37841, 35) (6104, 40)
(244, 147, 35) (244, 25, 40)
(244, 147, 35) (244, 25, 40)
4060
4060
(36395, 35) (6104, 40)
(37841, 35) (6104, 40)
(244, 147, 35) (244, 25, 40)
(244, 147, 35) (24

In [9]:
clip_idx = []
for i in range(len(subject_idx)):
    clip_idx.append(subject_idx[i]*1000+ time_idx[i])

In [10]:
face_all = np.concatenate(face_arrays, axis=0)
#mfcc_all = np.concatenate(mfcc_arrays, axis=0)
nirs_all = np.concatenate(nirs_arrays, axis=0)

# Can save the reshaped npy to local and call it directly from local 
np.save("npy/face_all",face_all)
np.save("npy/nirs_all",nirs_all)
np.save("npy/clip_idx",clip_idx)

In [ ]:
# Load npy from local
face_all = np.load("npy/face_all.npy", allow_pickle = True)
nirs_all= np.load("npy/nirs_all.npy", allow_pickle = True)
clip_idx = np.load("npy/clip_idx.npy", allow_pickle = True)

In [8]:
# verify the input shape
face_all.shape

(33997, 147, 35)